<a href="https://colab.research.google.com/github/Wozny614/mispy/blob/master/instalacja_pierwsze_cwiczenia_z_Pulp_Dariusz_Wo%C5%BAnica_215369.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


In [3]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

2.4


In [ ]:
pulp.pulpTestAll()

# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [6]:
from pulp import *
prob = LpProblem("The_Wiskas_Problem",LpMinimize)

x1 = LpVariable("ChickenPercent",0,None,LpInteger)
x2 = LpVariable("BeefPercent",0)

prob +=0.013*x1 + 0.008*x2,"Total_cost_of_Ingredients_per_can"

prob += x1 + x2 == 100,"PercentagesSum"
prob += 0.100*x1 + 0.200*x2 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.006*x2 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 <= 0.4, "SaltRequirement"









In [7]:
prob.writeLP("WhiskasModel.lp")

[BeefPercent, ChickenPercent]

In [8]:
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
  print(v.name,"=",v.varValue)

Status: Optimal
BeefPercent = 66.0
ChickenPercent = 34.0


In [9]:
print("Total_Cost_of_Ingredients_per_can = ", value(prob.objective))

Total_Cost_of_Ingredients_per_can =  0.97


# Whiskas more complex problem


In [5]:
from pulp import *

Ingredients = ['CHICKEN','BEEF','MUTTON','RICE','WHEAT','GEL','SARDINES']

costs = {'CHICKEN': 0.013,
         'BEEF':0.008,
         'MUTTON':0.010,
         'RICE':0.002,
         'WHEAT':0.005,
         'GEL':0.001,
         'SARDINES':0.001
         }


proteinPercent = {'CHICKEN': 0.100,
                  'BEEF':0.200,
                  'MUTTON':0.150,
                  'RICE':0.000,
                  'WHEAT':0.040,
                  'GEL':0.000,
         'SARDINES':0.1
                  }


fatPercent = {'CHICKEN': 0.080,
              'BEEF':0.100,
              'MUTTON':0.110,
              'RICE':0.010,
              'WHEAT':0.010,
              'GEL':0.000,
         'SARDINES':0.001
              }   


fibrePercent = {'CHICKEN': 0.001,
                  'BEEF':0.005,
                  'MUTTON':0.003,
                  'RICE':0.100,
                  'WHEAT':0.150,
                  'GEL':0.000,
         'SARDINES':0.001
                  }


saltPercent = {'CHICKEN': 0.002,
                  'BEEF':0.005,
                  'MUTTON':0.007,
                  'RICE':0.002,
                  'WHEAT':0.008,
                  'GEL':0.000,
         'SARDINES':0.006
                  }

vitaminBPercent = {'CHICKEN': 0.3e-8,
                  'BEEF':2.6e-8,
                  'MUTTON':2.6e-8,
                  'RICE':0.000,
                  'WHEAT':0.000,
                  'GEL':0.000,
                  'SARDINES':8.9e-8
                  }

prob = LpProblem("The_Whiskas_Problem",LpMinimize)
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)



prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total_Cost_of_Ingredients_per_can"


prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitaminBPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.24e-8, "VitaminBRequirement"



prob.solve()


variables_value = []
variables_name = []

for i in range(len(Ingredients)):
  variables_value.append(value(prob.variables()[i]))
  variables_name.append(Ingredients[i])
  
sklejone_name_value = [":".join((str(name), str(value))) for name,value in zip(sorted(variables_name),variables_value)]
sklejone_cale="%, ".join(sklejone_name_value)

print("Końcowy produkt będzie kosztować "+str(value(prob.objective))+", będzie miał w składzie: "+ str(sklejone_cale)+"%")

Końcowy produkt będzie kosztować 0.518823532, będzie miał w składzie: BEEF:59.831933%, CHICKEN:0.0%, GEL:23.361345%, MUTTON:0.0%, RICE:0.0%, SARDINES:16.806723%, WHEAT:0.0%
